In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from torch.utils.data import TensorDataset, DataLoader

In [3]:
import lightning as pl
from lightning.pytorch.loggers import TensorBoardLogger

import torchmetrics

In [18]:
# config
max_epochs = 60
learning_rate = 0.005

in_features=1152
h1 = 1000
h2 = 800
h3 = 500
out_features = 64
batch_size = 512

logging_name = 'Linear NN1'

project = "chess nn1 linear model 4, my moves, 1 label"
architecture = "Linear network with 2 hidden layers, my moves, 1 label",
dataset = "max rapid games",

In [5]:
data_training = torch.load('tensor_data/nn1_rapid_games_data_training_09_02_24.pt')
data_training = data_training.view(data_training.shape[0], 18, 8, 8)
# print(data_training.shape)
# data_training.to(device)

In [6]:
data_testing = torch.load('tensor_data/nn1_rapid_games_data_testing_09_02_24.pt')
data_testing = data_testing.view(data_testing.shape[0], 18, 8, 8)
# print(data_testing.shape)
# data_testing.to(device)

In [7]:
lables_training = torch.load('tensor_data/nn1_rapid_games_lables_training_09_02_24.pt')
# print(lables_training.shape)
# lables_training.to(device)

In [8]:
lables_testing = torch.load('tensor_data/nn1_rapid_games_lables_testing_09_02_24.pt')
# print(lables_testing.shape)
# lables_testing.to(device)

In [9]:
# data_training = data_training.half()
# data_testing = data_testing.half()
# lables_training = lables_training.half()
# lables_testing = lables_testing.half()
# lables_training = lables_training.float()
# lables_testing = lables_testing.float()

In [10]:
data_training = data_training.float()
data_testing = data_testing.float()
lables_training = lables_training.long()
lables_testing = lables_testing.long()

In [11]:
# print(data_testing.dtype)
# print(lables_training.dtype)
# print(lables_testing.dtype)

In [12]:
torch.manual_seed(42)

In [13]:
training_dataset = TensorDataset(data_training, lables_training)
testing_dataset = TensorDataset(data_testing, lables_testing)

In [14]:
data_loader_training = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=7)
data_loader_testing = DataLoader(testing_dataset, batch_size=batch_size, num_workers=7)


In [15]:
class Model(pl.LightningModule):

    def __init__(self, in_features=in_features, h1=h1, h2=h2, h3=h3, out_features=64): #out = 64 squares to move to 
        super().__init__()

        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        # self.fc2 = nn.Linear(h1, h2)
        # self.fc3 = nn.Linear(h2, h3)
        # self.fc3 = nn.Linear(h2, h3)
        self.out = nn.Linear(h2, out_features)

        self.accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=out_features)

    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        x = self.out(x)
        
        return x


    def training_step(self, batch, batch_index):

        x, y = batch
        preds = self.forward(x.view(len(y), -1))
        print(preds)


        loss = F.cross_entropy(preds, y)
        
        accuracy = self.accuracy(preds, y)
        # self.log('train_acc_step', self.accuracy)
        self.log('loss', loss)
        self.log('accuracy', accuracy)

        return loss

    def validation_step(self, batch, batch_index):

        x, y = batch
        preds = self.forward(x.view(len(y), -1))
        loss = F.cross_entropy(preds, y)
        accuracy = self.accuracy(preds, y)
        
        # self.log('train_acc_step', self.accuracy)
        self.log('loss validation', loss)
        self.log('accuracy validation', accuracy)

        return loss
        

    def configure_optimizers(self):

        return torch.optim.Adam(self.parameters(), lr=learning_rate)



In [16]:
model = Model()

In [19]:
logger = TensorBoardLogger("lightning_logs", name=logging_name) #tb logs is a directory?
# trainer = pl.Trainer(max_epochs=20, logger=logger, precision='32')
trainer = pl.Trainer(max_epochs=max_epochs, logger=logger, precision='16-mixed')

trainer.fit(model, data_loader_training, data_loader_testing)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/Linear NN1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 1.2 M 
1 | fc2  | Linear | 800 K 
2 | out  | Linear | 51.3 K
--------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
8.020     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

AttributeError: 'Model' object has no attribute 'accuracy'

In [ ]:
torch.save(model.state_dict(), f'models/nn1_model_ck1{conv1_kernels}_ck2{conv2_kernels}_fc1{l1}_fc2{l2}_e_{epochs}.pt')

In [ ]:
torch.save(Cnn, f'model_definitions/nn1_model_def_ck1{conv1_kernels}_ck2{conv2_kernels}_fc1{l1}_fc2{l2}_e_{epochs}.pt')

In [ ]:
torch.save(model, 'models/nn1_model_cnn_128_128_1624_1624_100_batches.pt')